In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../src/")

In [2]:
from package.logger import Timed, rlog, setup
from package import storage

setup("INFO")

In [3]:
import os

import folium
import geopandas as gpd
import pandas as pd

from package import strtime
from package.osm import osm
from package.mcr.data import NetworkType, OSMData
from package.mcr5.h3 import (
    get_h3_cells_for_nodes,
    get_h3_cells_for_bbox,
)
from package.h3 import plot_h3_cells_on_folium
from package.mcr5.h3_osm_interaction import get_location_mappings_for_cells
from package.mcr5.mcr5 import MCR5
from package.osm.osm import add_nearest_osm_node_id
from package.mcr5.labels import read_labels_for_nodes
from package.minute_city import minute_city

from package.geometa import GeoMeta
from package.mcr.data import NetworkType
import h3
from shapely.geometry import Polygon

In [4]:
city_name = "berlin"
geo_meta_path = f"../data/geometa_{city_name}.pkl"
city_id = "Berlin"
stops = f"../data/cleaned_{city_name}/stops.csv"
structs = f"../data/structs_{city_name}.pkl"
bicycle_location_path = "../data/bicycle_locations/2022-11-01_09_01_00.csv"

In [5]:
geo_meta = GeoMeta.load(geo_meta_path)
geo_data = OSMData(
    geo_meta,
    city_id,
    additional_network_types=[NetworkType.DRIVING],
)
nodes = geo_data.osm_nodes
edges = geo_data.osm_edges
driving_nodes, _, _ = geo_data.additional_networks[NetworkType.DRIVING]

[18:12:08] INFO     Using existing OSM data                                                              ]8;id=956386;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=673964;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (walking)                                              ]8;id=641534;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=399852;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

[18:13:00] INFO     Using existing OSM data                                                              ]8;id=707039;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=516311;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (driving)                                              ]8;id=723970;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=328085;file:///home/ppeter/repo/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

In [6]:
h3_cells = get_h3_cells_for_nodes(nodes[["lat", "lon"]].to_dict("records"), 9)
driving_h3_cells = get_h3_cells_for_nodes(
    driving_nodes[["lat", "lon"]].to_dict("records"), 9
)
h3_cells = h3_cells.intersection(driving_h3_cells)

In [7]:
residential_area = geo_meta.residential_area

In [8]:
CUTOFF = 0.25


def calculate_residential_intersection_area(h3_cell):
    boundary = h3.h3_to_geo_boundary(h3_cell)
    boundary_flipped = [(lon, lat) for lat, lon in boundary]
    poly = Polygon(boundary_flipped)
    intersection = poly.intersection(geo_meta.residential_area)
    return intersection.area / poly.area


h3_cells_filtered = {
    h3_cell
    for h3_cell in h3_cells
    if calculate_residential_intersection_area(h3_cell) > CUTOFF
}

In [9]:
m = folium.Map(location=geo_meta.get_center_lat_lon(), zoom_start=12)
geo_meta.add_to_folium_map(m)
plot_h3_cells_on_folium(h3_cells_filtered, m, popup_callback=lambda x, y: x)
m

In [10]:
location_mappings, invalid_h3_cells = get_location_mappings_for_cells(
    list(h3_cells_filtered), nodes, max_tries=20
)
invalid_h3_cells

[]

In [11]:
[lm.osm_node_id for lm in location_mappings if lm.h3_cell == "891fa188c4fffff"]

[2852959462]

In [12]:
car_location_mappings, invalid_h3_cells = get_location_mappings_for_cells(
    list(h3_cells_filtered), driving_nodes, max_tries=40
)
invalid_h3_cells

[]

In [13]:
storage.write_any_dict(
    {
        "location_mappings": location_mappings,
        "car_location_mappings": car_location_mappings,
    },
    "../data/location_mappings.pkl",
)